# 1. Data Ingestion and Exploration

This notebook covers the first step of the project: downloading the necessary raw data (SPC reports and NEXRAD files) and performing some initial exploratory analysis.

In [ ]:
import pandas as pd
import os
from datetime import datetime
import sys

# Add src directory to path to import our custom modules
sys.path.append('../src')
from data_processing import download_spc_reports, download_nexrad_data

## Step 1: Define Parameters and Paths

We'll focus on a specific date with known severe weather. A good case study helps in verifying the pipeline. Let's use a recent severe weather event in the DFW area.

In [ ]:
START_DATE = '20240528'
END_DATE = '20240528'
RADAR_SITE = 'KDFW' # Dallas/Fort Worth, a good site for severe weather
EVENT_TIME_UTC = datetime(2024, 5, 28, 6, 0) # 6:00 AM UTC on May 28th

RAW_DATA_PATH = '../data/raw/'
SPC_PATH = os.path.join(RAW_DATA_PATH, 'spc_reports')
NEXRAD_PATH = os.path.join(RAW_DATA_PATH, 'nexrad')

## Step 2: Download SPC Storm Reports

In [ ]:
download_spc_reports(START_DATE, END_DATE, SPC_PATH)

## Step 3: Explore the SPC Data

In [ ]:
report_file = os.path.join(SPC_PATH, f"{START_DATE[2:]}_rpts.csv")
spc_df = pd.read_csv(report_file)

# Filter for just hail reports
hail_reports = spc_df[spc_df['Type'] == 'Hail'].copy()

print(f"Found {len(hail_reports)} hail reports on {START_DATE}.")
hail_reports.head()

## Step 4: Download a NEXRAD Radar File

Let's download a single scan around the time of the event for initial analysis.

In [ ]:
download_nexrad_data(EVENT_TIME_UTC, RADAR_SITE, NEXRAD_PATH)

## Step 5: Load and Visualize the Radar Data with Py-ART

In [ ]:
import pyart
import matplotlib.pyplot as plt

# Find the downloaded file
downloaded_files = os.listdir(NEXRAD_PATH)
if downloaded_files:
    nexrad_file = os.path.join(NEXRAD_PATH, downloaded_files[0])
    print(f"Loading {nexrad_file}...")
    radar = pyart.io.read(nexrad_file)

    # Create a display object
    display = pyart.graph.RadarDisplay(radar)

    # Plot the base reflectivity
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    display.plot('reflectivity', sweep=0, ax=ax, vmin=-20, vmax=80, cmap='pyart_NWSRef')
    ax.set_title(f'Base Reflectivity for {RADAR_SITE} at {EVENT_TIME_UTC}')
    plt.show()
else:
    print("No NEXRAD files found in the directory.")